In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from src.utils import Norm01, Nback, Metr, MovingAverage, seconds_to_str 
from time import time

from scipy.linalg import hankel
import xgboost as xgb
from os import listdir

In [2]:
def getpred(df, edim=15): 
    pred=pd.DataFrame()  
    for i in range(1, edim):
        pred['data-'+str(i)]=df.Close.shift(i)
    pred.dropna(axis=0, inplace=True)
    pred.reset_index(drop=True, inplace=True)
    return pred

def forecasting(pred, bound, fwd, edim=15): 
    xgb_reg = xgb.XGBRegressor(n_estimators=20, max_depth=8)
    Xtrain=pred[:bound].values
    ytrain=df.Close[edim:bound+edim].values
    xgb_reg.fit(Xtrain, ytrain)
    ytest=[]
    Xtest=pred[bound:bound+1].values
    for i in range(fwd):
        y=xgb_reg.predict(Xtest)
        ytest.append(y[0])
        Xtest=np.concatenate((y.reshape(1,1), Xtest), axis=1)
        Xtest=np.delete(Xtest, -1, axis=1)
    yreal=df.Close[bound:bound+fwd]
    return ytest



In [3]:
datadir='..\_DataSets\HugeStockMarket\Stocks'
dfiles=listdir(datadir)
len(dfiles)

In [6]:
fwd=14
edim=15
cols=['series','meanmape', 'mapegt10','std']
manyforecasts=pd.read_csv('mfc.csv') #pd.DataFrame(columns=cols)
strt=time()
for i, f in enumerate(dfiles):
    df=pd.read_csv(datadir+'\\'+f)
    if len(df)<730:
        continue
    else:
        res=pd.DataFrame(columns=cols)
        df.drop(df[df.index<len(df)-730].index, inplace=True)
        df.reset_index(drop=True, inplace=True)
        res['series']=[f[:-7]]
        pred=getpred(df, edim=15)
        mape=[]
        for bound in range(365, 730-fwd, 3):
            ytest=forecasting(pred, bound, fwd, edim=15)
            yreal=df.Close[bound:bound+fwd]
            mape.append(Metr(ytest, yreal)[2])
        res['meanmape']=[np.mean(mape)]
        res['mapegt5']=[np.where(np.array(mape)>10)[0].shape[0]/len(mape)]
        res['std']=[np.std(mape)]
        manyforecasts=pd.concat([manyforecasts, res], axis=0, ignore_index=True)
        if i%100==0:
            manyforecasts.to_csv('mfc.csv', index=False)
            print(i, seconds_to_str(time()-strt), 'mfc.csv saved.')  
manyforecasts.to_csv('mfc.csv', index=False)
manyforecasts

0 00:00:06 mfc.csv saved.
200 00:28:39 mfc.csv saved.
300 00:49:53 mfc.csv saved.
500 01:38:13 mfc.csv saved.
600 02:01:56 mfc.csv saved.
900 03:13:25 mfc.csv saved.
1000 03:36:42 mfc.csv saved.
1100 03:55:47 mfc.csv saved.


,series,meanmape,mapegt5,std
0,a,2.101461,0.025641,1.932276
1,aa,5.966188,0.170940,4.754590
2,aaba,2.414769,0.008547,1.899085
3,aac,8.142165,0.256410,6.615187
4,aal,4.876859,0.102564,2.964076
...,...,...,...,...
5117,zoes,6.515495,0.128205,8.422109
5118,ztr,1.406385,0.000000,1.123485
5119,zts,2.093324,0.000000,1.733497
5120,zumz,6.589254,0.213675,4.916093


In [7]:
manyforecasts.to_csv('mfc.csv', index=False)
len(mape)

107